# 1. 필요한 라이브러리 가져오기

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# 2. 데이터 가져오기

In [22]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

## 2.1 데이터 훑어보기

In [23]:
train.head()

,num,date_time,전력사용량(kWh),기온,풍속(m/s),습도(%),강수량(mm|6시간),일조(hr|),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0


In [24]:
test.head()

,num,date_time,기온,풍속(m/s),습도(%),강수량(mm|6시간),일조(hr|3시간),비전기냉방설비운영,태양광보유
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,NaN,NaN
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,NaN,NaN
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2.2 데이터 결측치 확인하기

In [25]:
train.isnull().sum().sort_values(ascending=False)

num            0
date_time      0
전력사용량(kWh)     0
기온             0
풍속(m/s)        0
습도(%)          0
강수량(mm|6시간)    0
일조(hr|)        0
비전기냉방설비운영      0
태양광보유          0
dtype: int64

In [26]:
test.isnull().sum().sort_values(ascending=False)

태양광보유          8456
강수량(mm|6시간)    8400
비전기냉방설비운영      7784
기온             6720
풍속(m/s)        6720
습도(%)          6720
일조(hr|3시간)     6720
num               0
date_time         0
dtype: int64

**결측치가 없는 train 데이터로 EDA를 진행하는 것으로 결정함**

# 3. 데이터 전처리하기

## 3.1 영문 컬럼명으로 변경하기

In [28]:
train.columns = ['num', 'date_time', 'target', 'temp', 'wind', 'humid', 'rain', 'sunny', 'cooler', 'solar']
train.head()

,num,date_time,target,temp,wind,humid,rain,sunny,cooler,solar
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0


## 3.2 날짜와 관련된 컬럼 추가하기

In [32]:
train['date_time'] = pd.to_datetime(train['date_time'])
train['date'] = train['date_time'].dt.date
train['day'] = train['date_time'].dt.day
train['month'] = train['date_time'].dt.month
train['hour'] = train['date_time'].dt.hour
train['weekday'] = train['date_time'].dt.weekday # 0: 월요일
train['dayofyear'] = train.date_time.dt.dayofyear            # 연 기준 몇일째(숫자)
train.head()

,num,date_time,target,temp,wind,humid,rain,sunny,cooler,solar,datetime,date,day,month,hour,weekday,dayofyear
0,1,2020-06-01 00:00:00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,2020-06-01 00:00:00,2020-06-01,1,6,0,0,153
1,1,2020-06-01 01:00:00,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0,2020-06-01 01:00:00,2020-06-01,1,6,1,0,153
2,1,2020-06-01 02:00:00,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0,2020-06-01 02:00:00,2020-06-01,1,6,2,0,153
3,1,2020-06-01 03:00:00,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0,2020-06-01 03:00:00,2020-06-01,1,6,3,0,153
4,1,2020-06-01 04:00:00,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0,2020-06-01 04:00:00,2020-06-01,1,6,4,0,153
